![](https://storage.googleapis.com/mvps1_24/logo01.jpg)

![](https://storage.googleapis.com/mvps1_24/line01_02.jpg)

## Introdução

Este trabalho consiste na construção de um pipeline de dados utilizando tecnologias na nuvem. O pipeline irá envolver a busca e **coleta** de dados armazenando-os na nuvem, a construção de um **modelo de dados** juntamente com um **Catálogo de Dados** contendo minimamente uma descrição detalhada dos dados e seus domínios, a **carga** dos dados para um Data Lake utilizando pipelines de ETL (Extração, Transformação e Carga) nesta plataforma Databricks, **análise** dos dados e no final uma **autoanálise**.

## Objetivo
O objetivo é ver quais atores e atrizes mais participaram de filmes premiados com Oscar, relacionando eles às produções cinematográficas às quais atuaram. Neste banco de dados as produções estão entre os anos 1960 até 2024. Um segundo objetivo seria comparar a pontuação IMDB, que é dada pelo público, com o número de oscars que um filme ganhou.

## Descrição do problema
Serão usadas originalmente duas bases de dados da Internet Movie Database (IMDb), cujos atributos são mostrados nas tabelas abaixo e no detalhamento. A primeira base de dados contém títulos de filmes produzidos entre os anos 1960 e 2024 com seus respectivos códigos. A segunda base de dados contém nomes de artistas com os respectivos códigos dos filmes em que atuaram. 

![](https://storage.googleapis.com/mvps1_24/tab_basedados2.jpg)

Irei trabalhar estas duas tabelas para alinha-las a este projeto, detalhado em seguida.

**Ao término deste projeto iremos poder responder aos seguintes questionamentos:**

- Quais foram os atores / atrizes que mais participaram de produções vencedoras de Oscars.
- Comparar a relação existente entre indicações e premiações ao Oscar de atores e atrizes atuantes.
- O comparar a pontuação IMDb e numero de oscars de filmes.














![](https://storage.googleapis.com/mvps1_24/line01_03b.jpg)


O IMDb é um banco de dados, é uma fonte confiável de informações sobre cinema e televisão e é o maior banco de dados de cinema do mundo. Tem cadastro gratuito de avaliações de filmes, séries, jogos e muitos outros conteúdos.

As classificações do IMDb são médias ponderadas das avaliações dos usuários, e não médias de dados brutos. Isso significa que nem todas as classificações têm o mesmo impacto na classificação final, pois depende do número de votos para cada filme. A pontuação do IMDb é baseada na avaliação de usuários registrados, que podem dar notas de 1 a 10 para filmes, séries e outros conteúdos. Essas notas são agregadas e resumidas em uma única classificação.

As _nominations_, são o número de indicações na IMDb e incluem indicações para diferentes prêmios, em categorias variadas, evidenciando o sucesso e a repercussão da produção, bem como as indicações para concorrer ao Oscar. 
As indicações do IMDb são baseadas nas avaliações dos usuários do site, e também na atividade deles na lista de observação, enquanto que a escolha dos indicados ao Oscar é feita por membros da Academia de Artes e Ciências Cinematográficas.

Curiosidade: um critério usado é que uma nota 8 ou superior significa que o filme é bom ou tem alta probabilidade de ser bom, uma nota entre 8 e 7 as coisas podem variar muito e abaixo de 7 o filme não é muito bom.


![](https://storage.googleapis.com/mvps1_24/linha.jpg)

### Detalhamento

Coletei as duas bases de dados do site da Internet Movie Database, sendo que uma delas carreguei no Google Cloud Storage.

https://datasets.imdbws.com/name.basics.tsv.gz
https://storage.googleapis.com/mvps1_24/world_imdb_movies_top_movies_per_year_t4.csv.gz

A primeira base de dados contém todos os títulos de filmes produzidos entre os anos 1960 e 2024 com seus respectivos códigos, ano de lançamento, duração, pontuação IMDb, número de votos, indicações ao Oscar e número de Oscars que o filme ganhou. A segunda base de dados contém um número bem grande de nomes de atores, o código do filme em que eles atuaram, o ano de nascimento e falecimento, a profissão principal deles. Nem todos estes atores participaram de alguma produção entre os anos 1960 e 2024. Uma parte deles está associada a um ou mais códigos de filmes em que atuaram naquele período.


####Projeto Conceitual
O projeto conceitual tem o seguinte diagrama abaixo. A primeira tabela contém nomes de atores com seus respectivos códigos e tb os códigos dos filmes em que atuaram. A segunda tabela contém titulo dos filmes entre 1960 e 2024 com seus respectivos códigos. Os códigos dos filmes nas duas tabelas são iguais.

![](https://storage.googleapis.com/mvps1_24/prj_conceitual3.jpg)

Em um filme pode ter atuado no mínimo um ator ou no máximo muitos atores. Por outro lado um ator pode ter atuado no minimo em um filme ou no máximo em vários filmes. Isto configura um **relacionamento de muitos para muitos (N:N)** e neste caso a regra nos manda criar uma terceira tabela relacionando as duas primeiras tabelas, resultando como abaixo:

- **Tabela Filme**: com os atributos [PK]**cod_filme** e titulo.
- **Tabela Ator**: com um identificador único (por exemplo, [PK]**cod_ator**) e o nome.
- **Tabela Atua** (Relacionamento): que contém os campos [FK]**cod_filme** e [FK]**cod_ator** para representar o relacionamento muitos-para-muitos entre filmes e atores. 

Esta separação evita a redundância, pois cada ator é cadastrado uma única vez, e o relacionamento é feito através de chaves estrangeiras.

####Projeto Lógico
Neste meu projeto porém, irei considerar que as tabelas não serão alteradas no futuro e que somente o relacionamento do ator com o filme é o que me interessa, isto me permite simplificar o modelo lógico usando apenas duas tabelas:

- **Tabela Filmes**: com **cod_filme**, título, ano, duração, pts_imdb, n_votos, indicações, n_oscars.
- **Tabela Atores**: preparada para ser a tabela de relacionamento, com dois atributos: **cod_filme**, referenciando o filme, e **nome_ator**.

![](https://storage.googleapis.com/mvps1_24/prj_logico3.jpg)

Neste modelo, o nome do ator aparece quantas vezes forem necessárias, para cada filme no qual ele atuou. 
Embora isso já represente a relação muitos-para-muitos, a desvantagem é que o nome dos atores pode ficar 
repetido em diversas linhas. Esse design funciona bem se realmente não formos alterar ou expandir as tabelas. Irei usar a técnica de JOIN, que seria a junção entre as duas tabelas e poder responder.

####Descrição dos dados, seus domínios, valores mínimos e máximos esperados.
Baseado nos atributos das base de dados, segue a tabela com os tipos de dados e valores:   

![](https://storage.googleapis.com/mvps1_24/tab_basedados3.jpg)

Todos os campo não aceitam valor nulo, devem ser NOT NULL, menos o campo duração, ele permite valor nulo.
![](https://storage.googleapis.com/mvps1_24/linha.jpg)

####Programação

In [0]:
import io               #Library fornece as ferramentas para leitura e escrita de arquivos.
import requests         #Library utilizada para fazer requisições HTTP.
import gzip             #Para a descompactação de arquivos.

from pyspark.sql import SparkSession    # para poder baixar o arquivo compactado
from pyspark.sql.functions import col   # para poder ordenar as colunas

No quadro 11 abaixo criei um dataframe para conter o arquivo com os dados dos atores. O arquivo está em um repositório da Internet Movie Database (IMDb) e é do tipo CSV e está compactado em **tsv.gz**. Foi necessário também especificar a codificação tipo UTF-8.

Todo o processo de **carga** e descompactação está documentado abaixo e é feito com Pyspark.

In [0]:
# Criar uma instância de SparkSession de nome spark
spark = SparkSession.builder.appName("LerArquivoGzipDeURL").getOrCreate()
# SparkSession.builder: é o construtor
# getOrCreate(): cria uma nova SparkSession ou retorna uma existente, caso já tenha sido criada previamente.

# URL do arquivo Gzip
url = "https://datasets.imdbws.com/name.basics.tsv.gz"

# Fazer o download do arquivo gzip usando requests
response = requests.get(url)

# Descompactar o arquivo no formato Gzip
with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as filedescomp:
    # Carregar os dados descompactados em um DataFrame do Spark
    # gzip.GzipFile: classe usada para ler ou escrever arquivos compactados no formato Gzip.
    # fileobj=io.BytesIO(response.content): cria um objeto de arquivo em memória, que contém os dados HTTP (response.content)

    data = filedescomp.read().decode("utf-8")
    rdd = spark.sparkContext.parallelize(data.splitlines())
    # O método parallelize pega o texto e distribui entre os nós do cluster do Spark para serem processados paralelamente.
    # O método splitlines() divide uma string em várias linhas (quebras de linhas \n) onde cada linha é um elemento.
    ator_spark_df = spark.read.csv(rdd, sep="\t", header=True)

    ator_spark_df.printSchema()

    #NOME DO DATAFRAME:     ator_spark_df


root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: string (nullable = true)
 |-- deathYear: string (nullable = true)
 |-- primaryProfession: string (nullable = true)
 |-- knownForTitles: string (nullable = true)



Análise deste dataframe: **ator_spark_df**

In [0]:
ator_spark_df = ator_spark_df.orderBy(col("nconst").asc())    

# Exibindo o conteúdo do DataFrame Spark na ordem ascendente
ator_spark_df.show(20)

+---------+-------------------+---------+---------+--------------------+--------------------+
|   nconst|        primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+-------------------+---------+---------+--------------------+--------------------+
|nm0000001|       Fred Astaire|     1899|     1987|actor,miscellaneo...|tt0072308,tt00504...|
|nm0000002|      Lauren Bacall|     1924|     2014|actress,soundtrac...|tt0037382,tt00752...|
|nm0000003|    Brigitte Bardot|     1934|       \N|actress,music_dep...|tt0057345,tt00491...|
|nm0000004|       John Belushi|     1949|     1982|actor,writer,musi...|tt0072562,tt00779...|
|nm0000005|     Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050986,tt00694...|
|nm0000006|     Ingrid Bergman|     1915|     1982|actress,producer,...|tt0034583,tt00381...|
|nm0000007|    Humphrey Bogart|     1899|     1957|actor,producer,mi...|tt0034583,tt00432...|
|nm0000008|      Marlon Brando|     1924|     2004|actor,dir

Visualizando as 1000 primeiras linhas dos dados acima, ordenados pelo código do ator (nconst) em modo ascendente, aparentemente estão limpos. No próximo passo vou inverter para a ordem descendente, para visualizar a outra ponta dos dados. 

In [0]:
ator_spark_df = ator_spark_df.orderBy(col("nconst").desc())    

# Exibindo o conteúdo do DataFrame Spark na ordem descendente
ator_spark_df.show(50)

+---------+--------------------+---------+---------+--------------------+--------------------+
|   nconst|         primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+--------------------+---------+---------+--------------------+--------------------+
|nm9993719|          Andre Hill|       \N|       \N|                  \N|                  \N|
|nm9993718|         Aayush Nair|       \N|       \N|     cinematographer|           tt8736744|
|nm9993717|  Harikrishnan Rajan|       \N|       \N|     cinematographer|           tt8736744|
|nm9993716|       Essias Loberg|       \N|       \N|                  \N|                  \N|
|nm9993714|   Romeo del Rosario|       \N|       \N|animation_departm...|tt11657662,tt1406...|
|nm9993713|       Sambit Mishra|       \N|       \N|     writer,producer|tt20319332,tt2784...|
|nm9993712|     Corny O'Connell|       \N|       \N|                  \N|                  \N|
|nm9993711|       David Gluzman|       \N|       \

####Qualidade dos dados deste primeiro set.
**Análise (ator_spark_df).**

**-** Neste primeiro dataset notam-se alguns defeitos. Pela coluna _primaryName_, que é o nome do ator, notamos que tem atores/atrizes que não estão associados a nenhum título de filme. Estas tuplas precisam ser eliminadas. Outro defeito é que para vários atores/atrizes não consta o ano de nascimento. Estas tuplas também precisam ser eliminadas.

**-** As duas colunas, _primaryProfession_ e _knownForTitles_ são multivaloradas. As colunas _primaryProfession_ e _deathYear_ não interessam para este projeto, então irei eliminá-las também. A coluna _knownForTitles_ contém os códigos dos filmes e irei separa-los colocando cada um deles em uma célula, de modo que o nome do ator estará associado às produções em que atuou e o nome dele se repetirá pelo número de produções em que participou.

**-** A coluna _nconst_ é o código de cada ator/atriz. Esta coluna também não interessa para este projeto e irei elimina-la.

No próximo passo farei o desmembramento da coluna multivalorada _knownForTitles_ para fazer um relacionamento ator/filmes.

In [0]:
# Na tabela com nomes dos atores, temos dados multivalorados, ou seja, em uma tupla com a célula contendo o nome 
# de um ator a coluna correspondente com o código das produções, contém vários códigos. Irei separar esses códigos
# para ficarem um em cada célula. 
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

# similar a: SQL SELECT nconst, primaryName, knownForTitles... - só que o conteúdo da coluna knownForTitles é separado por virgulas.
atores_rel_filmes_spark_df = ator_spark_df.select(ator_spark_df.nconst, ator_spark_df.primaryName, ator_spark_df.birthYear, explode(split(ator_spark_df.knownForTitles, ',')).alias("id_tt"))

atores_rel_filmes_spark_df.show(50)      

#NOME DO DATAFRAME:     atores_rel_filmes_spark_df

+---------+------------------+---------+----------+
|   nconst|       primaryName|birthYear|     id_tt|
+---------+------------------+---------+----------+
|nm9993719|        Andre Hill|       \N|        \N|
|nm9993718|       Aayush Nair|       \N| tt8736744|
|nm9993717|Harikrishnan Rajan|       \N| tt8736744|
|nm9993716|     Essias Loberg|       \N|        \N|
|nm9993714| Romeo del Rosario|       \N|tt11657662|
|nm9993714| Romeo del Rosario|       \N|tt14069590|
|nm9993714| Romeo del Rosario|       \N| tt2455546|
|nm9993713|     Sambit Mishra|       \N|tt20319332|
|nm9993713|     Sambit Mishra|       \N|tt27843798|
|nm9993713|     Sambit Mishra|       \N|tt27191658|
|nm9993713|     Sambit Mishra|       \N|tt10709066|
|nm9993712|   Corny O'Connell|       \N|        \N|
|nm9993711|     David Gluzman|       \N|        \N|
|nm9993710| Nestor Rudnytskyy|       \N|        \N|
|nm9993709|         Lu Bevins|       \N|tt17717854|
|nm9993709|         Lu Bevins|       \N|tt11772904|
|nm9993709| 

Com a tabela acima nos aproximamos do objetivo. Na coluna _primaryName_ tem o nome do ator e a coluna _knownForTitles_ foi desmembrada e criei outra coluna _id_tt_ com os o código dos filmes separados um a um. Agora cada código de filme está associado a um ator. 

In [0]:
atores_rel_filmes_spark_df.printSchema()

root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: string (nullable = true)
 |-- id_tt: string (nullable = false)



Vou criar agora outro dataframe para fazer a carga do segundo arquivo, contendo o nome das produções cinematográficas.

In [0]:
# URL do arquivo Gzip
url = "https://storage.googleapis.com/mvps1_24/world_imdb_movies_top_movies_per_year_t4.csv.gz"

# Fazer o download do arquivo gzip usando requests
response = requests.get(url)

# Descompactar o arquivo no formato Gzip
with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as filedescomp:
    # Carregar os dados descompactados em um DataFrame do Spark
    # gzip.GzipFile: classe usada para ler ou escrever arquivos compactados no formato Gzip.
    # fileobj=io.BytesIO(response.content): cria um objeto de arquivo em memória, que contém os dados HTTP (response.content)

    data = filedescomp.read().decode("utf-8")
    rdd = spark.sparkContext.parallelize(data.splitlines())
    # O método parallelize pega o texto e distribui entre os nós do cluster do Spark para serem processados paralelamente.
    # O método splitlines() divide uma string em várias linhas (quebras de linhas \n) onde cada linha é um elemento.
    filmes_spark_df = spark.read.csv(rdd, sep=";", header=True, inferSchema=True)

    filmes_spark_df = filmes_spark_df.orderBy(col("year").desc()) 

    # Exibindo o conteúdo do DataFrame Spark na ordem ascendente
    filmes_spark_df.show(50)

    #NOME DO DATAFRAME:     filmes_spark_df

+----------+--------------------+----+--------+-----------+------+----------+-----+
|        id|               title|year|duration|rating_imdb|  vote|nomination|oscar|
+----------+--------------------+----+--------+-----------+------+----------+-----+
| tt0327785|   The Killer's Game|2024|  1h 44m|         57| 93000|         0|    0|
|tt12930350|My Spy The Eterna...|2024|  1h 52m|         57| 10000|         0|    0|
|tt10094388|         The Windigo|2024|  1h 25m|         53|   576|         0|    0|
|tt10128846|         Megalopolis|2024|  2h 18m|         48| 30000|         0|    0|
|tt10171472|            Goodrich|2024|  1h 50m|         63| 28000|         0|    0|
|tt10236164|The Girl with the...|2024|   2h 3m|         75|   967|        16|    0|
|tt10243672|                 Joy|2024|  1h 55m|         71| 66000|         0|    0|
|tt10245072|         Salem's Lot|2024|  1h 54m|         56| 30000|         0|    0|
|tt10284944|             The Fix|2024|  1h 38m|         45|   855|         0

In [0]:
filmes_spark_df = filmes_spark_df.orderBy(col("year").asc()) 

# Exibindo o conteúdo do DataFrame Spark na ordem ascendente
filmes_spark_df.show(50)

+---------+--------------------+----+--------+-----------+-----+----------+-----+
|       id|               title|year|duration|rating_imdb| vote|nomination|oscar|
+---------+--------------------+----+--------+-----------+-----+----------+-----+
|tt0049360|         El impostor|1960|  1h 19m|         66|   28|         0|    0|
|tt0050971|            Sigfrido|1960|  1h 37m|         57|   82|         0|    0|
|tt0051536|     Devil's Partner|1960|  1h 14m|         52|  615|         0|    0|
|tt0051863|    Life Is a Circus|1960|  1h 24m|         78|  201|         0|    0|
|tt0051874|          La Llorona|1960|  1h 15m|         58|  157|         0|    0|
|tt0052555|The Vampire and t...|1960|  1h 25m|         56|  682|         0|    0|
|tt0052607|The Battle of the...|1960|  1h 24m|         66|18000|         0|    0|
|tt0052614|          Le bel �ge|1960|  1h 40m|         70|   78|         0|    0|
|tt0052635|          Mein Kampf|1960|  1h 51m|         76|  396|         0|    0|
|tt0052674|     

####Qualidade dos dados deste segundo set.
**Análise (filmes_spark_df).**

**-** Listei o dataframe acima com mais de 1000 linhas, tanto na ordem ascendente como na descendente e verifiquei que os dados estão tratados e não é preciso fazer ajustes.

In [0]:
filmes_spark_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- duration: string (nullable = true)
 |-- rating_imdb: integer (nullable = true)
 |-- vote: integer (nullable = true)
 |-- nomination: integer (nullable = true)
 |-- oscar: integer (nullable = true)



CRIAÇÃO DO DATABASE **BRONZE** E CARGA DE DADOS

In [0]:
%sql CREATE DATABASE IF NOT EXISTS Bronze;   -- Se não existir, será criado o DataBase Bronze, que irá guardar os dados "crus"

In [0]:
# Salvando o DataFrame actors_basics... no DataBase Bronze
atores_rel_filmes_spark_df.write.mode("overwrite").saveAsTable("Bronze.atores_rel_filmes")

In [0]:
# Salvando o DataFrame movies_info... no DataBase Bronze
filmes_spark_df.write.mode("overwrite").saveAsTable("Bronze.filmes")

In [0]:
%sql SELECT nconst AS cod_Ator, primaryName AS Nome_ator,birthYear AS Ano_nascimento, id_tt AS cod_Filme FROM Bronze.atores_rel_filmes ORDER BY nconst DESC LIMIT 60 

cod_Ator,Nome_ator,Ano_nascimento,cod_Filme
nm9993719,Andre Hill,\N,\N
nm9993718,Aayush Nair,\N,tt8736744
nm9993717,Harikrishnan Rajan,\N,tt8736744
nm9993716,Essias Loberg,\N,\N
nm9993714,Romeo del Rosario,\N,tt11657662
nm9993714,Romeo del Rosario,\N,tt14069590
nm9993714,Romeo del Rosario,\N,tt2455546
nm9993713,Sambit Mishra,\N,tt20319332
nm9993713,Sambit Mishra,\N,tt27843798
nm9993713,Sambit Mishra,\N,tt27191658


Ordenando a tabela acima pelo ano de nascimento do ator, vemos que tem artistas sem o ano de nascimento. Vemos também que tem atores que não estão associados a nenhum filme. Estas tuplas podem ser eliminandas retirando aquelas onde o _ano_ tenha outros caracteres ascii que não sejam números, e as produções que não contenham a letras 'tt' na sua composição, visto que o código de filmes começa sempre pelas letras 'tt'.

Outra observação é que a base de dados relaciona produções entre os anos 1960 e 2024, então vou considerar apenas as produções com atores nascidos 100 anos antes do primeiro premio, ou seja, entre 1860 e 2024.

In [0]:
%sql 
-- Retirando as tuplas onde o ano de nascimento contenham outros carateres que não sejam números.

SELECT nconst AS cod_Ator, primaryName AS Nome_ator, birthYear AS Ano_nascimento, id_tt AS cod_Filme FROM Bronze.atores_rel_filmes WHERE birthYear BETWEEN CHAR(48) AND CHAR(57);

cod_Ator,Nome_ator,Ano_nascimento,cod_Filme
nm2889095,Barbara Devil,1983,tt7050960
nm2889095,Barbara Devil,1983,tt1167409
nm2889095,Barbara Devil,1983,tt1316462
nm2889095,Barbara Devil,1983,tt1261878
nm2889076,Kazimierz Janecki,1925,tt1171264
nm2889076,Kazimierz Janecki,1925,tt0088188
nm2889076,Kazimierz Janecki,1925,tt0086224
nm2889076,Kazimierz Janecki,1925,tt21982996
nm2889060,Eerie Von,1964,tt0425141
nm2889060,Eerie Von,1964,tt1067090


In [0]:
%sql 
-- Retirada da tuplas em que o código do filme não começa com as letras 'tt'. 
-- Retirada das tuplas onde o ano de nascimento esteja fora do intervalo de 1860 e 2024 

SELECT nconst AS cod_Ator, primaryName AS Nome_ator, birthYear AS Ano_nascimento, id_tt AS cod_Filme FROM Bronze.atores_rel_filmes WHERE id_tt LIKE '%tt%' AND birthYear BETWEEN 1860 AND 2024 
GROUP BY cod_ator, primaryName, birthYear, id_tt
ORDER BY primaryName DESC
LIMIT 50

cod_Ator,Nome_ator,Ano_nascimento,cod_Filme
nm2513158,​Rebekah Brooks,1968,tt0199257
nm2513158,​Rebekah Brooks,1968,tt0981226
nm2513158,​Rebekah Brooks,1968,tt0181260
nm2513158,​Rebekah Brooks,1968,tt0983197
nm1188541,Þórðarson Garðar Bachmann,1986,tt0306763
nm1930089,Þórunn Magnea Magnúsdóttir,1945,tt9100822
nm1930089,Þórunn Magnea Magnúsdóttir,1945,tt0805576
nm1930089,Þórunn Magnea Magnúsdóttir,1945,tt8309886
nm1930089,Þórunn Magnea Magnúsdóttir,1945,tt1384929
nm1800527,Þórunn Lárusdóttir,1973,tt31862843


In [0]:
filmes_spark_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- duration: string (nullable = true)
 |-- rating_imdb: integer (nullable = true)
 |-- vote: integer (nullable = true)
 |-- nomination: integer (nullable = true)
 |-- oscar: integer (nullable = true)



In [0]:
%sql 
-- retirada das tuplas onde o código de filme está NULL
SELECT id AS cod_Filme, title AS Titulo_filme, year AS Ano_criacao, duration AS Duracao, rating_imdb AS Pontos_IMDb, vote AS num_Votos, nomination AS Indicacoes, oscar AS n_Oscars FROM Bronze.filmes 
WHERE duration IS NOT NULL 
ORDER BY title ASC
LIMIT 50

cod_Filme,Titulo_filme,Ano_criacao,Duracao,Pontos_IMDb,num_Votos,Indicacoes,n_Oscars
tt1699720,!Women Art Revolution,2010,1h 23m,69,264,0,0
tt10620868,#Alive,2020,1h 38m,63,50000,0,0
tt0068152,$,1971,2h 1m,63,3000,0,0
tt0060697,"$1,000 on the Black",1966,1h 42m,59,418,0,0
tt0061313,"$10,000 Blood Money",1967,1h 37m,61,749,0,0
tt0790799,$9.99,2008,1h 18m,67,35000,4,0
tt2614684,'71,2014,1h 39m,72,62000,31,2
tt7518786,'83,2021,2h 42m,75,42000,52,0
tt4417402,'85: The Greatest Team in Football History,2016,1h 30m,68,165,0,0
tt0065600,"'Ctyri vrazdy stac�, drahousku'",1971,1h 43m,74,835,0,0


Nas linhas 36 e 37 abaixo vou aplicar os mesmos filtros das consultas SQL feitas acima ao dataframe e em seguida atualizar o database. Vou também retirar a coluna de código do ator (_cod_Ator_).

In [0]:
# Retirada da tuplas em que o código do filme não começa com as letras 'tt'. 

atores_rel_filmes_spark_df = atores_rel_filmes_spark_df.filter((atores_rel_filmes_spark_df.id_tt.like('%tt%')))

In [0]:
# Retirada das tuplas onde o ano de nascimento esteja fora do intervalo de 1860 e 2024 

atores_rel_filmes_spark_df = atores_rel_filmes_spark_df.filter((atores_rel_filmes_spark_df["birthYear"] >= "1860") & (atores_rel_filmes_spark_df["birthYear"] <= "2024")).orderBy(col("primaryName").asc()).drop("nconst")

atores_rel_filmes_spark_df.show(50)


+--------------------+---------+----------+
|         primaryName|birthYear|     id_tt|
+--------------------+---------+----------+
|"Stone" Robert A ...|     1982| tt1648179|
|"Stone" Robert A ...|     1982| tt1872194|
|"Stone" Robert A ...|     1982| tt1980929|
|"Stone" Robert A ...|     1982| tt2392137|
|'Amarillo Slim' P...|     1928| tt0071269|
|'Amarillo Slim' P...|     1928| tt1483037|
|'Amarillo Slim' P...|     1928| tt1446673|
|'Amarillo Slim' P...|     1928|tt20871262|
|'Atlas' Ramachandran|     1942| tt1311658|
|'Atlas' Ramachandran|     1942| tt3433162|
|'Atlas' Ramachandran|     1942| tt0928194|
|'Atlas' Ramachandran|     1942| tt2333760|
|'Baby' Carmen De Rue|     1908| tt0004635|
|'Baby' Carmen De Rue|     1908| tt0007787|
|'Baby' Carmen De Rue|     1908| tt0008144|
|'Baby' Carmen De Rue|     1908| tt0003698|
|  'Baby' Paul Cullen|     1962| tt0275309|
|'Beatle' Bob Matonis|     1953| tt1327590|
|'Beatle' Bob Matonis|     1953| tt4741930|
|'Beatle' Bob Matonis|     1953|

No proximo passo irei eliminar a coluna do ano de nascimento dos atores.Neste momento não nos interessa mais, pois já retirei os atores que nasceram fora da faixa de 1860 a 2024, e a tabela irá se alinhar mais ao projeto.

In [0]:
atores_rel_filmes_spark_df = atores_rel_filmes_spark_df.drop("birthYear")

In [0]:
atores_rel_filmes_spark_df.printSchema()

root
 |-- primaryName: string (nullable = true)
 |-- id_tt: string (nullable = false)



CRIAÇÃO DO DATABASE **SILVER** E CARGA DE DADOS

In [0]:
%sql CREATE DATABASE IF NOT EXISTS Silver;  -- Cria o DataBase Silver apenas se não existir

In [0]:
atores_rel_filmes_spark_df.write.mode("overwrite").saveAsTable("Silver.atores_rel_filmes")
filmes_spark_df.write.mode("overwrite").saveAsTable("Silver.filmes")

In [0]:
%sql
SELECT COUNT(*)
FROM Silver.filmes

count(1)
33241


In [0]:
%sql SELECT primaryName AS Nome_ator ,id_tt AS cod_Filme FROM Silver.atores_rel_filmes
    ORDER BY primaryName ASC LIMIT 50

Nome_ator,cod_Filme
"""Stone"" Robert A Stone III",tt1648179
"""Stone"" Robert A Stone III",tt1872194
"""Stone"" Robert A Stone III",tt1980929
"""Stone"" Robert A Stone III",tt2392137
'Amarillo Slim' Preston,tt0071269
'Amarillo Slim' Preston,tt1483037
'Amarillo Slim' Preston,tt1446673
'Amarillo Slim' Preston,tt20871262
'Atlas' Ramachandran,tt3433162
'Atlas' Ramachandran,tt0928194


Irei criar o database Gold e copiar as tabelas do DB Silver para o DB Gold, para finalmente fazer os questionamentos propostos pelo projeto.

CRIAÇÃO DO DATABASE **GOLD** E CARGA DE DADOS

In [0]:
%sql CREATE DATABASE IF NOT EXISTS Gold;  -- Cria o DataBase Gold apenas se não existir

In [0]:
filmes_spark_df.write.mode("overwrite").saveAsTable("Gold.filmes");
atores_rel_filmes_spark_df.write.mode("overwrite").saveAsTable("Gold.atores_rel_filmes");

In [0]:
atores_rel_filmes_spark_df.printSchema()

root
 |-- primaryName: string (nullable = true)
 |-- id_tt: string (nullable = false)



In [0]:
filmes_spark_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- duration: string (nullable = true)
 |-- rating_imdb: integer (nullable = true)
 |-- vote: integer (nullable = true)
 |-- nomination: integer (nullable = true)
 |-- oscar: integer (nullable = true)



Verificamos acima que os atributos das duas tabelas estão de acordo com o projeto.

![](https://storage.googleapis.com/mvps1_24/linha.jpg)

####Solução do problema

**Respostas aos questionamentos propostos.**
![](https://storage.googleapis.com/mvps1_24/linha.jpg)

Na linha baixo vou fazer a primeira consulta: fornescendo alguns nomes de atores e atrizes para ver quem ganhou mais oscar e pontos.

**1)** Quais foram os atores e atrizes que mais participaram de produções que ganharam Oscars?

Vou escolher os seguintes atores: Jack Nicholson, Tom Cruise, Kevin Bacon, Meryl Streep, Anthony Hopkins, Emma Stone, Kate Winslet e Nicole Kidman.

In [0]:
%sql 
SELECT primaryName AS Nome_ator, title AS Titulo_filme, year AS Ano, rating_imdb AS Pontos_IMDb, vote AS n_Votos, nomination AS Indicacoes, oscar AS n_Oscars FROM Gold.atores_rel_filmes 
INNER JOIN Gold.filmes ON Gold.atores_rel_filmes.id_tt = Gold.filmes.id 
WHERE primaryName IN ("Jack Nicholson", "Tom Cruise", "Kevin Bacon", "Meryl Streep", "Anthony Hopkins", "Emma Stone", "Kate Winslet", "Nicole Kidman")
ORDER BY primaryName 
LIMIT 50

Nome_ator,Titulo_filme,Ano,Pontos_IMDb,n_Votos,Indicacoes,n_Oscars
Anthony Hopkins,The Silence of the Lambs,1991,86,16000000,50,0
Anthony Hopkins,The World's Fastest Indian,2005,78,61000,6,0
Anthony Hopkins,Hitchcock,2012,68,80000,29,1
Anthony Hopkins,Nixon,1995,71,33000,18,4
Emma Stone,La La Land,2016,80,703000,307,0
Emma Stone,Easy A,2010,70,427000,22,0
Emma Stone,The Help,2011,81,506000,121,0
Emma Stone,Poor Things,2023,78,320000,414,0
Jack Nicholson,Chinatown,1974,81,360000,24,0
Jack Nicholson,As Good as It Gets,1997,77,325000,54,0


**Comentário**: Na consulta acima foi usado a técnica de JOIN, uma junção entre as duas tabelas. Ordenando pelo número de Oscar notamos que, entre estes artistas escolhidos, o ator Kevin Bacon participou mais vezes de produções premiadas com Oscar, ao passo que o filme Poor Things teve 414 indicações, mas não ganhou Oscars. Lembro que as _indicações_ incluem indicações para diferentes prêmios, em categorias variadas.

![](https://storage.googleapis.com/mvps1_24/linha.jpg)
**2)** Quais foram os atores / atrizes que mais participaram de produções com mais indicações ao Oscar? Lembrando que o intervalo de premiações está entre os anos 1960 e 2024.

In [0]:
%sql 
SELECT primaryName AS Nome_ator, title AS Titulo_filme, year AS Ano, duration AS Duracao, (rating_imdb / 10) AS Pontos_IMDb, vote AS n_Votos,nomination AS Indicacoes, oscar AS n_Oscars FROM Gold.atores_rel_filmes 
INNER JOIN Gold.filmes ON Gold.atores_rel_filmes.id_tt = Gold.filmes.id 
WHERE oscar > 0
ORDER BY oscar DESC
LIMIT 50


Nome_ator,Titulo_filme,Ano,Duracao,Pontos_IMDb,n_Votos,Indicacoes,n_Oscars
Carl Anderson,The Color Purple,1985,2h 34m,7.7,100000,25,11
Steve Ray,The Color Purple,1985,2h 34m,7.7,100000,25,11
Paul Whiteman,The Color Purple,1985,2h 34m,7.7,100000,25,11
Snooky Young,The Color Purple,1985,2h 34m,7.7,100000,25,11
Marianna Tcherkassky,The Turning Point,1977,1h 59m,6.8,53000,18,11
Speckled Red,The Color Purple,1985,2h 34m,7.7,100000,25,11
Porter Grainger,The Color Purple,1985,2h 34m,7.7,100000,25,11
Starr Danias,The Turning Point,1977,1h 59m,6.8,53000,18,11
Adolph Caesar,The Color Purple,1985,2h 34m,7.7,100000,25,11
Thomas A. Dorsey,The Color Purple,1985,2h 34m,7.7,100000,25,11


Comentário: Temos acima a lista dos atores e atrizes (listei 5000) que mais participaram de filmes premiados, mas se observa outra coisa interessante. Pelo critério popular citado no início, os filmes com pontuação IMDb entre 7 e 8, são considerados neutros. Ordenando pela colna de número de Oscars nota-se que a grande maioria dos filmes que ganharam mais Oscar tem pontuação IMDb neutra. Isto pode ser devido por exemplo a altas expectativas sobre o filme. Filmes que ganham muitos Oscars geralmente vêm com uma grande expectativa. Pode ser também a diferença de público. O IMDb reflete a opinião de um público global e diversificado, enquanto os Oscars são decididos por um grupo específico de votantes da indústria cinematográfica.

![](https://storage.googleapis.com/mvps1_24/linha.jpg)
**3)** Comparar a pontuação IMDb e número de oscars de filmes. Qual a coerência entre a pontuação e a premiação do Oscar.

In [0]:
%sql 
SELECT title AS Titulo_filme, duration AS Duracao, (rating_imdb / 10) AS Pontos_IMDb, vote AS n_Votos, nomination AS Indicacoes, oscar AS n_Oscars FROM Gold.atores_rel_filmes 
INNER JOIN Gold.filmes ON Gold.atores_rel_filmes.id_tt = Gold.filmes.id  
WHERE rating_imdb > 0 AND oscar > 0
GROUP BY title, duration, rating_imdb, vote, nomination, oscar
ORDER BY rating_imdb DESC 
LIMIT 50

Titulo_filme,Duracao,Pontos_IMDb,n_Votos,Indicacoes,n_Oscars
The Shawshank Redemption,2h 22m,9.3,3000000,42,7
Fight Club,2h 19m,8.8,24000000,38,1
The Green Mile,3h 9m,8.6,15000000,37,4
Se7en,2h 7m,8.6,19000000,44,1
City of God,2h 10m,8.6,824000,50,4
American History X,1h 59m,8.5,12000000,15,1
The Prestige,2h 10m,8.5,15000000,44,2
For Sama,1h 40m,8.5,13000,52,1
Psycho,1h 49m,8.5,741000,14,4
Spider-Man: Across the Spider-Verse,2h 20m,8.5,428000,164,1


Comentário: Na consulta acima confirmamos o comentário da consulta anterior. Outro motivo desta diferença pode ser devido aos temas e estilo. Muitos filmes premiados no Oscar têm temas artísticos, históricos ou dramáticos que podem não agradar a todos os públicos. Enquanto críticos e a Academia valorizam esses aspectos, o público geral pode achar o filme "difícil" ou "lento", resultando em avaliações mais neutras.


![](https://storage.googleapis.com/mvps1_24/linha.jpg)

In [0]:
%sql 
SELECT title AS Titulo_filme, duration AS Duracao, oscar AS n_Oscars, (rating_imdb / 10) AS Pontos_IMDB,
    CASE 
        WHEN rating_imdb > 80 THEN 'Bom'
        WHEN rating_imdb < 70 THEN 'Ruim'
        ELSE 'Neutro'
    END AS Classif
FROM Gold.filmes 
WHERE rating_imdb > 0 
ORDER BY oscar DESC 
LIMIT 50

Titulo_filme,Duracao,n_Oscars,Pontos_IMDB,Classif
The Turning Point,1h 59m,11,6.8,Ruim
The Color Purple,2h 34m,11,7.7,Neutro
Gangs of New York,2h 47m,10,7.5,Neutro
True Grit,1h 50m,10,7.6,Neutro
American Hustle,2h 18m,10,7.2,Neutro
Killers of the Flower Moon,3h 26m,10,7.6,Neutro
The Banshees of Inisherin,1h 54m,9,7.7,Neutro
The Sand Pebbles,3h 2m,8,7.5,Neutro
The Elephant Man,2h 4m,8,8.2,Bom
Ragtime,2h 35m,8,7.3,Neutro


Nesta última consulta, a título de curiosidade, acrescentei uma coluna que classifica o filme como Bom, Neutro ou Ruim segundo o critério descrito na introdução.  

![](https://storage.googleapis.com/mvps1_24/linha.jpg)

####Autoavaliação.
A proposta deste projeto foi mostrar de maneira símples, um pipeline de dados envolvendo a busca, coleta, modelagem, carga e análise dos dados utilizando tecnologias na nuvem. Foi feita a coleta e carga usando Pyspark e o Data Lake da Google Cloud Storage. Incluí também o processo de criação dos databases Bronze, Silver e Gold, onde o DB Bronze recebeu os dados crus, até o DB Gold onde foi finalizado o processo. Poderiam ter sido usadas outras library adicionais do Python, como o Pandas para formatar todas as tabelas, mas meu conhecimento sobre esta library ainda não me favorece na velocidade de produção, porém estou me atualizando sobre isto. 

Devido à minha total falta de experiência sobre esse universo de bancos de dados, estou conhecendo isto somente agora, precisei reestruturar todo o pipeline duas vezes. A primeira vez foi porque eu tinha feito uma coleta de um banco proveniente de uma base de dados da IMDb (ok), mas a segunda base de dados eu tinha armazenado no Databrick. Então achei que não seria o ideal e resolvi deslocar esta base de dados do Databricks para o Data Lake da Google Cloud Storage. A segunda vez que decidi refazer o pipeline foi porque notei que os nomes de várias tabelas não estavam padronizados, então refiz o procedimento todo para renomear as tabelas e os dataframes.

Com tudo, o objetivo foi alcançado. 
![](https://storage.googleapis.com/mvps1_24/linha.jpg)
 